In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-midterm-p2/train.csv
/kaggle/input/2023-ml-midterm-p2/test.csv
/kaggle/input/2023-ml-midterm-p2/submit_sample.csv


In [2]:
# 시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
# 파일 불러오기
train = pd.read_csv('/kaggle/input/2023-ml-midterm-p2/train.csv')
test = pd.read_csv('/kaggle/input/2023-ml-midterm-p2/test.csv')
submit = pd.read_csv('/kaggle/input/2023-ml-midterm-p2/submit_sample.csv')

In [4]:
# 학습데이터에서 클래스 라벨 분리
X = train.drop('target', axis=1)
y = train['target']

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trestbps  242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalach   242 non-null    int64  
 9   exang     242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slope     242 non-null    int64  
 12  ca        242 non-null    object 
 13  thal      242 non-null    object 
dtypes: float64(1), int64(11), object(2)
memory usage: 26.6+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     61 non-null     int64  
 1   age       61 non-null     int64  
 2   sex       61 non-null     int64  
 3   cp        61 non-null     int64  
 4   trestbps  61 non-null     int64  
 5   chol      61 non-null     int64  
 6   fbs       61 non-null     int64  
 7   restecg   61 non-null     int64  
 8   thalach   61 non-null     int64  
 9   exang     61 non-null     int64  
 10  oldpeak   61 non-null     float64
 11  slope     61 non-null     int64  
 12  ca        61 non-null     object 
 13  thal      61 non-null     object 
dtypes: float64(1), int64(11), object(2)
memory usage: 6.8+ KB


In [7]:
# object 자료형인 컬럼에 어떤 값이 있는지 살펴본다
X['ca'].unique()

array(['0', '3', '2', '1', '?'], dtype=object)

In [8]:
X['thal'].unique()

array(['6', '3', '7', '?'], dtype=object)

In [9]:
# X[(X['ca']=='?')] = -1
# test[(test['ca']=='?')] = -1

In [10]:
# X[(X['thal']=='?')] = -1
# test[(test['thal']=='?')] = -1

In [11]:
# 숫자가 아닌값들을 숫자로 바꿔주기 위해 LabelEncoder을 불러온다
from sklearn.preprocessing import LabelEncoder
index = ['thal', 'ca']
for i in index:
    le = LabelEncoder()
    X[i] = le.fit_transform(X[i])
    test[i] = le.transform(test[i])

In [12]:
# 결측값을 채우기 위한 Simple
# from sklearn.impute import SimpleImputer
# imp = SimpleImputer(missing_values='?', strategy='constant')
# X = imp.fit_transform(X['thal'], X['ca'])
# test['thal'] = imp.transform(test['thal'])

In [13]:
# X['ca'] = X['ca'].astype('64')
# test['ca'] = test['ca'].astype('int64')
# X['thal'] = X['thal'].astype('int64')
# test['thal'] = test['thal'].astype('int64')

In [14]:
# 정규화를 위한 standardscaler import
from sklearn.preprocessing import MinMaxScaler
std = MinMaxScaler()
X_std = std.fit_transform(X)
test_std = std.transform(test)

In [15]:
# LogisticRegression과 판별분석 모델 불러오고 학습
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#model = QuadraticDiscriminantAnalysis(reg_param=0.4)
model = LogisticRegression(C=1, max_iter=10000)
model.fit(X_std, y)

LogisticRegression(C=1, max_iter=10000)

In [16]:
#예측 수행
pred = model.predict(test_std)
submit['target'] = pred
submit.to_csv("submit.csv", index=False)